# 🤖 Model Training: Random Forest for Daily Energy Prices

In [ ]:
# 📦 Libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
import seaborn as sns


## 1. Load & Prepare Data

In [ ]:
# Load preprocessed data (after lag, rolling, etc.)
df = pd.read_csv("your_preprocessed_data.csv", parse_dates=["Datetime"], index_col="Datetime")
df = df.dropna()  # drop NA rows caused by shifting/rolling

# Features and target
features = ['lag_1', 'lag_7', 'rolling_mean_7', 'rolling_std_7', 'dayofweek', 'month', 'dayofyear']
target = 'MarginalES'
X = df[features]
y = df[target]


## 2. Train/Validation/Test Split (Chronological)

In [ ]:
# Chronological split: 70% train, 15% val, 15% test
split_1 = int(len(df) * 0.7)
split_2 = int(len(df) * 0.85)
X_train, y_train = X.iloc[:split_1], y.iloc[:split_1]
X_val, y_val = X.iloc[split_1:split_2], y.iloc[split_1:split_2]
X_test, y_test = X.iloc[split_2:], y.iloc[split_2:]


## 3. Train Random Forest Regressor

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


## 4. Evaluation

In [ ]:
def evaluate(y_true, y_pred, label="Set"):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    print(f"{label} MAE: {mae:.2f}")
    print(f"{label} RMSE: {rmse:.2f}")
    return mae, rmse

# Validation
y_val_pred = model.predict(X_val)
evaluate(y_val, y_val_pred, label="Validation")

# Test
y_test_pred = model.predict(X_test)
evaluate(y_test, y_test_pred, label="Test")


## 5. Plot Results

In [ ]:
plt.figure(figsize=(14,6))
plt.plot(y_test.index, y_test, label='Actual', marker='o')
plt.plot(y_test.index, y_test_pred, label='Predicted', marker='x')
plt.title("Random Forest Prediction vs Actual")
plt.xlabel("Date")
plt.ylabel("€/MWh")
plt.legend()
plt.grid(True)
plt.tight_layout()


## ✅ Next: Hyperparameter tuning or model comparison